In [1]:
import cobra
import optlang
import mcs.mcs_computation as mcs_computation
import numpy

In [2]:
ecc2 = cobra.io.read_sbml_model("ECC2comp.sbml")
# allow all reactions that are not boundary reactions as cuts (same as exclude_boundary_reactions_as_cuts option of compute_mcs)
cuts = numpy.array([not r.boundary for r in ecc2.reactions])
reac_id = ecc2.reactions.list_attr('id') # list of reaction IDs in the model
# define target (multiple targets are possible; each target can have multiple linear inequality constraints)
ecc2_mue_target = [[("Growth", ">=", 0.01)]] # one target with one constraint, a.k.a. syntehtic lethals
# this constraint alone would not be sufficient, but there are uptake limits defined in the reaction bounds
# of the model that are integerated below, first, however, parse the constraint(s) and convert to matrix format
ecc2_mue_target = [mcs_computation.relations2leq_matrix(
                   mcs_computation.parse_relations(t, reac_id_symbols=mcs_computation.get_reac_id_symbols(reac_id)), reac_id)
                   for t in ecc2_mue_target]
# now integrate the non-default reaction bounds from the network
mcs_computation.integrate_model_bounds(ecc2, ecc2_mue_target)
# this is just to show what the first (and only) target now looks like:
for c in mcs_computation.get_leq_constraints(ecc2, ecc2_mue_target)[0]:
    print(c)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-09-03
c3b14102-43c0-11ec-8cb3-e45e37029bdb: -1.0*Growth + 1.0*Growth_reverse_699ae <= -0.01
c3b14103-43c0-11ec-86e9-e45e37029bdb: 1.0*AcUp - 1.0*AcUp_reverse_36302 <= 10.0
c3b14104-43c0-11ec-af39-e45e37029bdb: 1.0*CO2Up - 1.0*CO2Up_reverse_9ac6c <= 10.0
c3b14105-43c0-11ec-9c61-e45e37029bdb: 1.0*GlcUp - 1.0*GlcUp_reverse_f2a50 <= 10.0
c3b14106-43c0-11ec-ad31-e45e37029bdb: 1.0*GlycUp - 1.0*GlycUp_reverse_d1488 <= 10.0
c3b14107-43c0-11ec-b5b8-e45e37029bdb: 1.0*SuccUp - 1.0*SuccUp_reverse_79869 <= 10.0


In [3]:
# calculate MCS up to size 3 with "any MCS" enumeration method
ecc2_mcs,_ = mcs_computation.compute_mcs(ecc2, ecc2_mue_target, cuts=cuts, enum_method=3, max_mcs_size=3, network_compression=True,                                              include_model_bounds=False) # include_model_bounds is disabled because it already has been done above
print(len(ecc2_mcs), "MCS found.")
# show MCS as n-tuples of reaction IDs
ecc2_mcs_rxns= [tuple(reac_id[r] for r in mcs) for mcs in ecc2_mcs]
print(ecc2_mcs_rxns)
# check that all MCS disable the first (and only) target
print(all(mcs_computation.check_mcs(ecc2, ecc2_mue_target[0], ecc2_mcs, optlang.interface.INFEASIBLE)))

Running FVA to find blocked reactions...
0.0779999999795109
Read LP format model from file C:\Users\phili\AppData\Local\Temp\tmprna021kv.lp
Reading time = 0.01 seconds
: 94 rows, 244 columns, 834 nonzeros
Flipped EX_ca2_ex
Flipped EX_cl_ex
Flipped EX_cobalt2_ex
Flipped EX_cu2_ex
Flipped EX_fe2_ex
Flipped EX_fe3_ex
Flipped EX_glc_DASH_D_ex
Flipped EX_glyc_ex
Flipped EX_k_ex
Flipped EX_mg2_ex
Flipped EX_mn2_ex
Flipped EX_mobd_ex
Flipped EX_nh4_ex
Flipped EX_ni2_ex
Flipped EX_o2_ex
Flipped EX_pi_ex
Flipped EX_so4_ex
Flipped EX_zn2_ex
Flipped RPI
Removing 5 reactions:
[EX_q8_c, EX_adp_c, EX_nad_c, EX_mqn8_c, EX_nadp_c]
Running FVA for desired regions...
Using big M.


C:\Users\phili\.conda\envs\cnapy-dev\lib\site-packages\scipy\sparse\sputils.py:114: UserWarning: object dtype is not supported by sparse matrices
  warnings.warn("object dtype is not supported by sparse matrices")


Objective function is empty; set objective to self.minimize_sum_over_z
Found solution with objective value 2.0
CS (64, 66) -> MCS (64, 66)
Found solution with objective value 3.0
CS (5, 6, 7) -> MCS (5, 6, 7)
Found solution with objective value 3.0
CS (7, 20, 21) -> MCS (7, 21)
Found solution with objective value 3.0
CS (11, 14, 33) -> MCS (11, 33)
Found solution with objective value 3.0
CS (15, 33, 63) -> MCS (15,)
Found solution with objective value 2.0
CS (42, 52) -> MCS (42, 52)
Found solution with objective value 1.0
CS (17,) -> MCS (17,)
Found solution with objective value 3.0
CS (45, 56, 64) -> MCS (56, 64)
Found solution with objective value 3.0
CS (11, 24, 67) -> MCS (11, 24)
Found solution with objective value 2.0
CS (29, 52) -> MCS (29, 52)
Found solution with objective value 3.0
CS (60, 62, 63) -> MCS (60, 62)
Found solution with objective value 2.0
CS (28, 29) -> MCS (28, 29)
Found solution with objective value 1.0
CS (57,) -> MCS (57,)
Found solution with objective value 

In [4]:
# %% same calculation without network compression
ecc2_mcsF,_ = mcs_computation.compute_mcs(ecc2, ecc2_mue_target, cuts=cuts, enum_method=3, max_mcs_size=3, network_compression=False,
                                        include_model_bounds=False)
print(set(ecc2_mcs) == set(ecc2_mcsF)) # check that results agree

Running FVA to find blocked reactions...
0.09400000001187436
Found 5 blocked reactions:
 ['EX_adp_c', 'EX_mqn8_c', 'EX_nad_c', 'EX_nadp_c', 'EX_q8_c']
Running FVA for desired regions...
Using big M.
Objective function is empty; set objective to self.minimize_sum_over_z
Found solution with objective value 3.0
CS (119, 120, 121) -> MCS (119, 120)
Found solution with objective value 3.0
CS (55, 70, 121) -> MCS (55, 70)
Found solution with objective value 3.0
CS (51, 52, 79) -> MCS (52, 79)
Found solution with objective value 3.0
CS (70, 102, 120) -> MCS (70, 102)
Found solution with objective value 1.0
CS (83,) -> MCS (83,)
Found solution with objective value 3.0
CS (73, 102, 121) -> MCS (73, 102, 121)
Found solution with objective value 3.0
CS (55, 61, 102) -> MCS (55, 102)
Found solution with objective value 2.0
CS (110, 119) -> MCS (110,)
Found solution with objective value 2.0
CS (42, 43) -> MCS (42, 43)
Found solution with objective value 3.0
CS (70, 73, 121) -> MCS (70, 73, 121)
Fou